In [1]:
from skimage.measure import compare_ssim
import cv2
import numpy as np

# Dem Netzwerk werden nicht die gesamten Frames zum trainieren gegeben, sondern nur die Regionen in denen sich etwas
# geändert hat.
def apply_region_filter(before, after):
    before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
    after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)
    (score, diff) = compare_ssim(before_gray, after_gray, full=True)
    diff = (diff * 255).astype("uint8")
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    before_region = np.zeros(before.shape, np.uint8)
    after_region = np.zeros(after.shape, np.uint8)

    for c in contours:
        area = cv2.contourArea(c)
        if area > 40:
            x, y, w, h = cv2.boundingRect(c)
            before_region[y:y + h, x:x + w] = before[y:y + h, x:x + w]
            after_region[y:y + h, x:x + w] = after[y:y + h, x:x + w]

    return before_region, after_region

In [2]:
# Es werden 2 "Inputs" benötigt => eigene Implementierung eines DataGenerators für Keras
def generate_generator_multiple(generator, dir1, dir2, batch_size, img_height, img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size=(img_height, img_width),
                                          class_mode='categorical',
                                          batch_size=batch_size,
                                          shuffle=True,
                                          seed=666)

    genX2 = generator.flow_from_directory(dir2,
                                          target_size=(img_height, img_width),
                                          class_mode='categorical',
                                          batch_size=batch_size,
                                          shuffle=True,
                                          seed=666)

    while True:
        X1i = genX1.next()
        X2i = genX2.next()

        before = np.reshape(X1i[0], (584, 480, 3))
        after = np.reshape(X2i[0], (584, 480, 3))
        before, after = apply_region_filter(before.copy(), after.copy())

        before = np.asarray(before)
        before = before.astype('float32')
        before /= 255.0

        after = np.asarray(before)
        after = after.astype('float32')
        after /= 255.0

        before = np.reshape(before, (1, 584, 480, 3))
        after = np.reshape(after, (1, 584, 480, 3))
        yield [before, after], X2i[1]
        

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

in_channels = 3
img_height, img_width = 584, 480  # original
num_classes = 2  # click or no click
batch_size = 1
epochs = 1

train_dir_1 = "dataset/train/before"
train_dir_2 = "dataset/train/after"

test_dir_1 = "dataset/test/before"
test_dir_2 = "dataset/test/after"

input_shape = (img_height, img_width, in_channels)

generator = ImageDataGenerator(rescale=1, #1./255,
                               rotation_range=0,
                               width_shift_range=0,
                               height_shift_range=0,
                               shear_range=0,
                               zoom_range=0,
                               horizontal_flip=False,
                               vertical_flip=False,
                               #fill_mode='nearest'
                               )

inputgenerator = generate_generator_multiple(generator=generator,
                                             dir1=train_dir_1,
                                             dir2=train_dir_2,
                                             batch_size=batch_size,
                                             img_height=img_height,
                                             img_width=img_width)

testgenerator = generate_generator_multiple(generator,
                                            dir1=test_dir_1,
                                            dir2=test_dir_2,
                                            batch_size=batch_size,
                                            img_height=img_height,
                                            img_width=img_width)

In [4]:
# Erzeugen der gewünschten Architektur (auf Basis von https://xin-xia.github.io/publication/icse193.pdf)
before_input = layers.Input(shape=input_shape)
base_model_before = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    pooling='max',
    input_shape=input_shape,
    weights='imagenet',
    input_tensor=before_input
)

after_input = layers.Input(shape=input_shape)
base_model_after = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    pooling='max',
    input_shape=input_shape,
    weights='imagenet',
    input_tensor=after_input
)

for layer in base_model_after.layers:
    layer._name = layer.name + str("_2")

conv = layers.concatenate([base_model_before.output, base_model_after.output])
conv = layers.Flatten()(conv)

dense = layers.Dense(512)(conv)
dense = layers.LeakyReLU(alpha=0.1)(dense)
dense = layers.Dropout(0.5)(dense)

output = layers.Dense(num_classes, activation='softmax')(dense)

model = Model(inputs=[before_input, after_input], outputs=[output])

In [ ]:
# Training des Softmax-Layers
from tensorflow.keras.callbacks import ModelCheckpoint

opt = optimizers.Adam()
# opt = optimizers.SGD(lr=0.001, decay=0.0001, momentum=0.9, nesterov=True)

for layer in base_model_before.layers:
    layer.trainable = False

for layer in base_model_after.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['categorical_accuracy'])
best_weights_file = "weights.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

history = model.fit_generator(inputgenerator,
                              steps_per_epoch=404/batch_size,
                              epochs=epochs,
                              validation_data=testgenerator,
                              validation_steps=102/batch_size,
                              #use_multiprocessing=True,
                              shuffle=False,
                              verbose=1,
                              callbacks=callbacks)

Found 405 images belonging to 2 classes.
Found 405 images belonging to 2 classes.
 23/404 [>.............................] - ETA: 6:50 - loss: 5.5446 - categorical_accuracy: 0.6522

/home/gattma/gattma/workspaces/aktT1-click-detector/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  # Remove the CWD from sys.path while we load stuff.
/home/gattma/gattma/workspaces/aktT1-click-detector/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  # Remove the CWD from sys.path while we load stuff.
/home/gattma/gattma/workspaces/aktT1-click-detector/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  # Remove the CWD from sys.path while we load stuff.
/

In [ ]:
# Training inkl. der CNN-Schichten
for layer in base_model_before.layers:
    layer.trainable = True

for layer in base_model_after.layers:
    layer.trainable = True

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['categorical_accuracy'])
history = model.fit_generator(inputgenerator,
                              steps_per_epoch=404 / batch_size,
                              epochs=epochs,
                              validation_data=testgenerator,
                              validation_steps=102 / batch_size,
                              # use_multiprocessing=True,
                              shuffle=False,
                              verbose=1,
                              callbacks=callbacks)

Found 405 images belonging to 2 classes.
Found 405 images belonging to 2 classes.
  3/404 [..............................] - ETA: 24:44 - loss: 6.6950 - categorical_accuracy: 0.3333

/home/gattma/gattma/workspaces/aktT1-click-detector/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  # Remove the CWD from sys.path while we load stuff.
/home/gattma/gattma/workspaces/aktT1-click-detector/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  # Remove the CWD from sys.path while we load stuff.
/home/gattma/gattma/workspaces/aktT1-click-detector/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  # Remove the CWD from sys.path while we load stuff.
/